# data1.7.csv
## TF-IDF
# 

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import numpy as np

data = pd.read_csv('data1.7.csv')
data.head()

,Place,Details,Class
0,"""The Song of Everlasting Sorrow""","['song', 'everlasting', 'sorrow', 'historical'...",None Related
1,25th of April Bridge,"['april', '25', 'bridge', 'spanned', 'sides', ...",None Related
2,25th of April Bridge,"['april', '25', 'bridge', 'spanned', 'sides', ...",None Related
3,3D Black Art,"['myeongdong', '3d', 'art', 'museum', 'uses', ...",None Related
4,3billion,"['company', 'offers', 'genetic', 'testing', 'r...",bioinformatic


In [2]:
def mat_to_df(mat, vec):
    df = pd.DataFrame(mat.toarray(), columns=vec.get_feature_names_out())
    return df

In [3]:
# Create a TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')  # Default English stop words 

# Fit and transform the 'Details' column
tfidf_matrix = tfidf_vectorizer.fit_transform(data['Details'])
# Convert the TF-IDF matrix to a DataFrame
tfidf_df = mat_to_df(tfidf_matrix, tfidf_vectorizer)
tfidf_df.head()

,007,0115,095,10,100,1000,1005,100th,101,1023,...,zones,zoo,zoological,zoom,zug,zun,zurich,zwingli,zyt,évora
0,0.0,0.0,0.0,0.0,0.0,0.108162,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# Concatenate the TF-IDF DataFrame with the original DataFrame
result_df = pd.concat([data, tfidf_df], axis=1)

# Display the resulting DataFrame
print(result_df.head(3))

                              Place  \
0  "The Song of Everlasting Sorrow"   
1              25th of April Bridge   
2              25th of April Bridge   

                                             Details         Class  007  0115  \
0  ['song', 'everlasting', 'sorrow', 'historical'...  None Related  0.0   0.0   
1  ['april', '25', 'bridge', 'spanned', 'sides', ...  None Related  0.0   0.0   
2  ['april', '25', 'bridge', 'spanned', 'sides', ...  None Related  0.0   0.0   

   095   10  100      1000  1005  ...  zones  zoo  zoological  zoom  zug  zun  \
0  0.0  0.0  0.0  0.108162   0.0  ...    0.0  0.0         0.0   0.0  0.0  0.0   
1  0.0  0.0  0.0  0.000000   0.0  ...    0.0  0.0         0.0   0.0  0.0  0.0   
2  0.0  0.0  0.0  0.000000   0.0  ...    0.0  0.0         0.0   0.0  0.0  0.0   

   zurich  zwingli  zyt  évora  
0     0.0      0.0  0.0    0.0  
1     0.0      0.0  0.0    0.0  
2     0.0      0.0  0.0    0.0  

[3 rows x 11728 columns]


# use result_df
## Naive bayes
# 

In [5]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.utils import resample
import imblearn
from imblearn.combine import SMOTETomek

# From previous DataFrame
selected = result_df.copy()

# Split the data into training and testing sets
train_data, test_data = train_test_split(selected, test_size=0.2, random_state=42)

# TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(train_data['Details'])
X_test_tfidf = tfidf_vectorizer.transform(test_data['Details'])

X_train_df = mat_to_df(X_train_tfidf, tfidf_vectorizer)
X_test_df = mat_to_df(X_test_tfidf, tfidf_vectorizer)

# Handling imbalance class / data
smt = SMOTETomek(random_state=42)
X_res, y_res = smt.fit_resample(X_train_tfidf, train_data["Class"])
print(y_res.value_counts())

# Create and train a Naive Bayes classifier
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_res, y_res)


# Model Evaluation
# Make Predictions on the test set
predictions = naive_bayes_classifier.predict(X_test_tfidf)

# Evaluate the classifier
accuracy = accuracy_score(test_data['Class'], predictions)
conf_matrix = confusion_matrix(test_data['Class'], predictions)
classification_rep = classification_report(test_data['Class'], predictions)

# Display evaluation metrics
print(f'Accuracy: {accuracy:.2f}')
print('\nConfusion Matrix:')
print(conf_matrix)
print('\nClassification Report:')
print(classification_rep)

# Save the train model to a file
joblib.dump(naive_bayes_classifier, 'model.h5')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')

Class
robotic          704
None Related     704
biotechnology    704
bioinformatic    704
aerospace        704
Name: count, dtype: int64
Accuracy: 0.99

Confusion Matrix:
[[178   0   0   0   0]
 [  0   8   0   0   1]
 [  0   0   8   0   0]
 [  0   0   1   8   0]
 [  0   0   0   0  23]]

Classification Report:
               precision    recall  f1-score   support

 None Related       1.00      1.00      1.00       178
    aerospace       1.00      0.89      0.94         9
bioinformatic       0.89      1.00      0.94         8
biotechnology       1.00      0.89      0.94         9
      robotic       0.96      1.00      0.98        23

     accuracy                           0.99       227
    macro avg       0.97      0.96      0.96       227
 weighted avg       0.99      0.99      0.99       227



['tfidf_vectorizer.pkl']

In [6]:
# Load the saved TF-IDF vectorizer
loaded_vectorizer = joblib.load('tfidf_vectorizer.pkl')
# Load the saved Naive Bayes classifier
naive_bayes_classifier = joblib.load('model.h5')


def class_det(Details, vectorizer, classifier):
    input_data = [Details]
    vectorized_input_data = vectorizer.transform(input_data)
    predictions = classifier.predict(vectorized_input_data)
    print(predictions)
    return predictions
    


In [7]:
queries = [
    'Astra makes engines for spacecraft using electric propulsion technology and provides launch services for space missions. Astra reached a key milestone in 2021, succeeding in its first commercial launch to low Earth orbit. The company says its launch systems are designed to be cost efficient with a payload capacity of approximately 600 kg',
    "Sinopharm is known for developing two inactivated virus COVID-19 vaccines. One of them is the Sinopharm BIBP COVID-19 vaccine, also known as BBIBP-CorV2. This vaccine completed Phase III trials in several countries with over 60,000 participants. It’s one of the vaccines being used in vaccination campaigns by certain countries in Asia, Africa, South America, and Europe",
    "Magic Leap, Inc. is an American technology company that released a head-mounted augmented reality display, called Magic Leap One, which superimposes 3D computer-generated imagery over real world objects. It is attempting to construct a light-field chip using silicon photonics.",
    "The rockets are very versatile. They can reach up to 10000 lbs of thrust and reach velocities to escape Earth's orbit. They will be essential in reaching the moon.",
]


for query in queries:
    print(query, class_det(query, loaded_vectorizer, naive_bayes_classifier))
    print()

['aerospace']
Astra makes engines for spacecraft using electric propulsion technology and provides launch services for space missions. Astra reached a key milestone in 2021, succeeding in its first commercial launch to low Earth orbit. The company says its launch systems are designed to be cost efficient with a payload capacity of approximately 600 kg ['aerospace']

['biotechnology']
Sinopharm is known for developing two inactivated virus COVID-19 vaccines. One of them is the Sinopharm BIBP COVID-19 vaccine, also known as BBIBP-CorV2. This vaccine completed Phase III trials in several countries with over 60,000 participants. It’s one of the vaccines being used in vaccination campaigns by certain countries in Asia, Africa, South America, and Europe ['biotechnology']

['robotic']
Magic Leap, Inc. is an American technology company that released a head-mounted augmented reality display, called Magic Leap One, which superimposes 3D computer-generated imagery over real world objects. It is a